In [21]:
import numpy as np
import pandas as pd
from timeit import default_timer as timer
from datetime import timedelta
from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.parsing.preprocessing import remove_stopwords
import scipy

movies = pd.read_csv('movie_archive\movies_metadata.csv')
ratings_small = pd.read_csv('movie_archive\\ratings_small.csv')
links_small = pd.read_csv('movie_archive\links_small.csv')


C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
import nltk
from nltk.stem import PorterStemmer
words = set(nltk.corpus.words.words())
porter = PorterStemmer()

def remove_non_english(sent):
    s = " ".join(w for w in nltk.wordpunct_tokenize(sent) if w.lower() in words or not w.isalpha())
    return s



In [5]:
df = movies.filter(['id','title','overview'], axis=1)

In [6]:
# Create new movies df, with only important columns
df = movies.filter(['id','title','overview'], axis=1)
print('Before: ' + str(len(df)))
df = df.dropna()
# check movies ids, convert to int, drop if not possible
# check if text in english
for id in df['id'].to_list():
    try:
        id = int(id[0])
    except ValueError:
        df = df[df['id'] != id]

for title in df['title'].to_list():
    try:
        title = str(title)
        if title == 'nan' or title == '' or title == ' ':
            df = df[df['title'] != title]
        elif len(remove_non_english(title)) < len(title)/4:
            df = df[df['title'] != title]
    except ValueError:
        df = df[df['title'] != title]

for overview in df['overview'].to_list():
    try:
        overview = str(overview)
        if overview == 'nan' or overview == '' or overview == ' ' or overview == '...':
            df = df[df['overview'] != overview]
        elif len(remove_non_english(overview)) < len(overview)/2:
            df = df[df['overview'] != overview]
    except ValueError:
        df = df[df['overview'] != overview]

df = df.reset_index(drop=True)

print('After: ' + str(len(df)))

Before: 45466
After: 37713


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37713 entries, 0 to 37712
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        37713 non-null  object
 1   title     37713 non-null  object
 2   overview  37713 non-null  object
dtypes: object(3)
memory usage: 884.0+ KB


In [103]:
movie_texts = df['overview'].apply(str).to_list()
movie_titles = df['title'].apply(str).to_list()
text_tokens = []
for i in range(len(movie_texts)):
    if movie_titles[i] != 'nan' and movie_texts[i] != 'nan':
        text = movie_titles[i] + " " + movie_texts[i]
        #text = porter.stem(text)
        text = remove_stopwords(text)
        text_tokens.append(gensim.utils.simple_preprocess(text))

tagged_text = [TaggedDocument(t, [i]) for i, t in enumerate(text_tokens)]

In [23]:
text_len = []
for i in range(len(tagged_text)):
    text_len.append(len(tagged_text[i].words))

avg_len = sum(text_len) / len(text_len)
avg_len

32.25383819902951

In [104]:
model = gensim.models.doc2vec.Doc2Vec(dm=0, dbow_words=1, min_count=4, negative=3,
                hs=0, sample=1e-4, window=10, vector_size=100, workers=8)
#model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=3, window=5 ,epochs=10)

model.build_vocab(tagged_text)

model.train(tagged_text, total_examples=model.corpus_count, epochs=model.epochs)

In [98]:
model.wv.most_similar('good')

[('natured', 0.7135628461837769),
 ('slob', 0.5717315077781677),
 ('so', 0.5646793842315674),
 ('charm', 0.5600972175598145),
 ('bad', 0.5483207106590271),
 ('awfully', 0.5448116660118103),
 ('tuned', 0.5418558716773987),
 ('sione', 0.5400313138961792),
 ('doing', 0.536138117313385),
 ('easygoing', 0.5302959680557251)]

In [13]:
ratings_train, ratings_test = train_test_split(ratings_small, test_size=0.2)
print(len(ratings_train))
print(len(ratings_test))

80003
20001


In [14]:
movieid_list = df['id'].to_list()
movieid_list = list(map(int, movieid_list))


for r in ratings_train['movieId']:
    try:
        mid = r
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == int(mid)]
        index = movieid_list.index(int(tmdbId))
    except ValueError:
        ratings_train = ratings_train[ratings_train['movieId'] != mid]

print(len(ratings_train))


68157


In [15]:
for r in ratings_test['movieId']:
    try:
        mid = r
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == int(mid)]
        index = movieid_list.index(int(tmdbId))
    except ValueError:
        ratings_test = ratings_test[ratings_test['movieId'] != mid]

print(len(ratings_test))

16963


In [48]:
ratings = ratings_small
for r in ratings['movieId']:
    try:
        mid = r
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == int(mid)]
        index = movieid_list.index(int(tmdbId))
    except ValueError:
        ratings = ratings[ratings['movieId'] != mid]

print(len(ratings))

85120


In [105]:
inferr_doc_vecs = []
for i in range(len(tagged_text)):
    inferred_vector = model.infer_vector(tagged_text[0].words, steps=30, alpha=0.025)
    inferr_doc_vecs.append(inferred_vector)

In [106]:
# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}

#userid_list = ratings_train['userId'].unique()
userid_list = ratings['userId'].unique()
movieid_list = df['id'].to_list()
movieid_list = list(map(int, movieid_list))

# get all movie vector rated from user
def get_user_rated_movies(ids):
    
        movie_vector_list = []
        for i in ids:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
            try:
                index = movieid_list.index(int(tmdbId))
                #movie_vector = model.dv[index]
                movie_vector = inferr_doc_vecs[index]
                movie_vector_list.append(movie_vector)
            except ValueError:
                print("ValueError")
                #pass

        movie_vectors = movie_vector_list
        return movie_vectors

for uid in userid_list:
    #user_profile = ratings_train.loc[ratings_train['userId'] == uid]
    user_profile = ratings.loc[ratings['userId'] == uid]
    user_rated_movies_vector_list = get_user_rated_movies(user_profile['movieId'].to_list())
    user_rated_movies_vector_array = np.array(user_rated_movies_vector_list)
    #user_vector = user_rated_movies_vector_array.sum(axis=0)
    user_vector = np.mean(user_rated_movies_vector_array, axis=0)
    all_user_profiles[uid] = user_vector

In [45]:
len(all_user_profiles)

671

In [13]:
def get_user_rated_movies_bitarray(ids):
    
        bitarray = np.zeros(len(movieid_list))
        for i in ids:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
            try:
                index = movieid_list.index(int(tmdbId))
                bitarray[index] = 1
            except ValueError:
                #print("ValueError")
                pass

        return bitarray


all_user_bitarrays = {}

for uid in userid_list:
    user_profile = ratings_train.loc[ratings_train['userId'] == uid]
    user_rated_movies_bitarray = get_user_rated_movies_bitarray(user_profile['movieId'].to_list())
    all_user_bitarrays[uid] = user_rated_movies_bitarray


In [14]:
len(all_user_bitarrays)

671

In [15]:
doc_vecs = model.dv.get_normed_vectors()
len(doc_vecs)
type(doc_vecs)

numpy.ndarray

In [16]:
import pickle
with open('doc2vec_vecsize_50_user_profiles.pickle', 'wb') as f:
    pickle.dump(all_user_profiles,f, protocol=pickle.HIGHEST_PROTOCOL)

with open('doc2vec_vecsize_50_doc_vecs.pickle', 'wb') as f:
    pickle.dump(model.dv.get_normed_vectors(),f, protocol=pickle.HIGHEST_PROTOCOL)

with open('doc2vec_vecsize_50_bit_output.pickle', 'wb') as f:
    pickle.dump(all_user_bitarrays,f, protocol=pickle.HIGHEST_PROTOCOL)


In [17]:
#Test Data

# compute all movie vector form specific user into one vector for each user
all_user_profiles_test = {}

userid_list = ratings_test['userId'].unique()
movieid_list = df['id'].to_list()
movieid_list = list(map(int, movieid_list))

# get all movie vector rated from user
def get_user_rated_movies(ids):
    
        movie_vector_list = []
        for i in ids:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
            try:
                index = movieid_list.index(int(tmdbId))
                movie_vector = model.dv[index]
                movie_vector_list.append(movie_vector)
            except ValueError:
                #print("ValueError")
                pass

        movie_vectors = movie_vector_list
        return movie_vectors

for uid in userid_list:
    user_profile = ratings_test.loc[ratings_test['userId'] == uid]
    user_rated_movies_vector_list = get_user_rated_movies(user_profile['movieId'].to_list())
    user_rated_movies_vector_array = np.array(user_rated_movies_vector_list)
    #user_vector = user_rated_movies_vector_array.sum(axis=0)
    user_vector = np.mean(user_rated_movies_vector_array, axis=0)
    all_user_profiles[uid] = user_vector

In [18]:
# Test Data

def get_user_rated_movies_bitarray(ids):
    
        bitarray = np.zeros(len(movieid_list))
        for i in ids:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
            try:
                index = movieid_list.index(int(tmdbId))
                bitarray[index] = 1
            except ValueError:
                #print("ValueError")
                pass

        return bitarray


all_user_bitarrays = {}

for uid in userid_list:
    user_profile = ratings_test.loc[ratings_test['userId'] == uid]
    user_rated_movies_bitarray = get_user_rated_movies_bitarray(user_profile['movieId'].to_list())
    all_user_bitarrays[uid] = user_rated_movies_bitarray

In [19]:
with open('doc2vec_vecsize_50_user_profiles_test.pickle', 'wb') as f:
    pickle.dump(all_user_profiles_test,f, protocol=pickle.HIGHEST_PROTOCOL)

with open('doc2vec_vecsize_50_bit_output_test.pickle', 'wb') as f:
    pickle.dump(all_user_bitarrays,f, protocol=pickle.HIGHEST_PROTOCOL)

In [107]:
recommendation_users = {}
for uid in userid_list:
    if uid in all_user_profiles.keys():
        sims = model.dv.most_similar([all_user_profiles[uid]], topn=len(model.dv))
        recommendation_users[uid] = sims
len(recommendation_users)

671

In [108]:
%%time

percentage_already_seen = []
for uid in userid_list:
    user_profile = ratings_train.loc[ratings_train['userId'] == uid]
    user_recommendation = recommendation_users[uid]

    user_profile_mid_list = user_profile['movieId'].to_list()
    user_m_index_list = []
    for m in user_profile_mid_list:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
        try:
            index = movieid_list.index(int(tmdbId))
            user_m_index_list.append(index)
        except ValueError:
            #print("ValueError: " + str(m))
            pass
    
    m_count = 0
    for m_i, cos in user_recommendation[:len(user_profile)]:
        try:
            if int(m_i) in user_m_index_list:
                m_count += 1
        except ValueError:
            #print("ValueError: " + str(m_i) + " not in tmdbid list") 
            pass  

    if m_count == len(user_profile):
        print(str(uid) + " : the first movies are already viewed")
    else:
        print(str(m_count) + " of " + str(len(user_profile)) + " in first recommended already seen")

    p = m_count/len(user_profile)
    percentage_already_seen.append(p)

average_percentage = sum(percentage_already_seen) / len(percentage_already_seen)
print("Average Percentage of already seen movies: " + str(average_percentage))

0 of 15 in first recommended already seen
0 of 43 in first recommended already seen
0 of 32 in first recommended already seen
3 of 139 in first recommended already seen
2 of 69 in first recommended already seen
1 of 35 in first recommended already seen
2 of 60 in first recommended already seen
1 of 78 in first recommended already seen
1 of 27 in first recommended already seen
0 of 40 in first recommended already seen
0 of 28 in first recommended already seen
0 of 43 in first recommended already seen
4 of 33 in first recommended already seen
1 of 19 in first recommended already seen
38 of 1154 in first recommended already seen
0 of 21 in first recommended already seen
1 of 245 in first recommended already seen
0 of 32 in first recommended already seen
2 of 277 in first recommended already seen
1 of 69 in first recommended already seen
1 of 109 in first recommended already seen
2 of 149 in first recommended already seen
8 of 465 in first recommended already seen
0 of 15 in first recommen

In [125]:
#%%time
# Average Rank of Movies -> User Vector

def compute_avg_rank(uid):
    user_profile = ratings_train.loc[ratings_train['userId'] == uid]
    user_recommendation = recommendation_users[uid]

    user_profile_mid_list = user_profile['movieId'].to_list()
    user_m_index_list = []
    for m in user_profile_mid_list:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
        try:
            index = movieid_list.index(int(tmdbId))
            user_m_index_list.append(index)
        except ValueError:
            #print("ValueError: " + str(m))
            pass
    
    rank_list = []
    for mid in user_m_index_list:
        rank = 0
        for m_i, cos in user_recommendation:
            try:
                if int(m_i) == int(mid):
                    break
                rank = rank + 1
            except ValueError:
                pass
        
        rank_list.append(rank)   
    

    r = sum(rank_list)/len(user_profile)
    avg_ranks.append(r)
    #print("UserId: " + str(uid) + "Avg_Rank: " + str(r) + "User_Len: " + str(len(user_profile)))



In [126]:
avg_ranks = []

start = timer()

with ThreadPoolExecutor(max_workers=8) as executer:
    executer.map(compute_avg_rank, userid_list)

end = timer()
print ("Time Calculate Vectors HH:MM:SS: ",timedelta(seconds=end-start))

print("Max Rank = " + str(len(df)))
average_percentage = sum(avg_ranks) / len(avg_ranks)
print("Average Rank of already seen movies: " + str(average_percentage))

Time Calculate Vectors HH:MM:SS:  0:09:05.243834
Max Rank = 37713
Average Rank of already seen movies: 19928.398176487823


In [39]:
# Remove movies used to create User Vector

percentage_already_seen = []
cleaned_recommendations = {}
for uid in userid_list:
    
    user_profile = ratings_train.loc[ratings_train['userId'] == uid]
    user_recommendation = recommendation_users[uid]
    user_profile_mid_list = user_profile['movieId'].to_list()
    user_m_index_list = []
    for m in user_profile_mid_list:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
        try:
            index = movieid_list.index(int(tmdbId))
            user_m_index_list.append(index)
        except ValueError:
            pass
            
    m_count = 0
    for m_i, cos in user_recommendation:
        try:
            if int(m_i) in user_m_index_list:
                m_count += 1
                user_recommendation.remove((m_i,cos))
        except ValueError:
            pass  

    cleaned_recommendations[uid] = user_recommendation

In [40]:
# Calculate Percentage of user rated movies from test split in recommended, top 10/50/100
top_value = 100
user_percentages = []
userid_list_test = ratings_test['userId'].unique()
for uid in userid_list_test:
    if uid in all_user_profiles.keys():
        user_profile = ratings_test.loc[ratings_test['userId'] == uid]
        found_movies = 0
        if uid in recommendation_users.keys():
            user_recommendations = cleaned_recommendations[uid]
            #print(len(user_recommendations))
            #tmdbId_list = user_profile['tmdbId'].to_list():
            user_profile_movie_list = user_profile['movieId'].to_list()
            user_m_index_list = []
            for m in user_profile_movie_list:
                tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
                try:
                    index = movieid_list.index(int(tmdbId))
                    user_m_index_list.append(index)
                except ValueError:
                    #print("ValueError: " + str(tmdbId))
                    pass

            for m_i, cos in user_recommendations[:top_value]:
                try:
                    if int(m_i) in user_m_index_list:
                        found_movies = found_movies + 1
                except ValueError:
                    pass
            
            print(str(found_movies) + " of " + str(len(user_profile)) + " found in topn= " + str(top_value))

        
        #user_percentages.append(found_movies/top_value)
        user_percentages.append(found_movies/len(user_profile))

mean_percentage = sum(user_percentages)/len(user_percentages)
print("Mean Percentage of good Recommendations (based on Test Split): " + str(mean_percentage))

0 of 33 found in topn= 100
0 of 11 found in topn= 100
0 of 56 found in topn= 100
0 of 9 found in topn= 100
0 of 26 found in topn= 100
0 of 194 found in topn= 100
0 of 9 found in topn= 100
0 of 29 found in topn= 100
0 of 10 found in topn= 100
1 of 213 found in topn= 100
2 of 406 found in topn= 100
0 of 111 found in topn= 100
0 of 97 found in topn= 100
0 of 54 found in topn= 100
0 of 83 found in topn= 100
0 of 27 found in topn= 100
0 of 23 found in topn= 100
0 of 164 found in topn= 100
1 of 31 found in topn= 100
0 of 55 found in topn= 100
0 of 29 found in topn= 100
1 of 93 found in topn= 100
2 of 311 found in topn= 100
0 of 41 found in topn= 100
0 of 36 found in topn= 100
0 of 39 found in topn= 100
0 of 11 found in topn= 100
1 of 41 found in topn= 100
0 of 8 found in topn= 100
0 of 33 found in topn= 100
0 of 42 found in topn= 100
0 of 28 found in topn= 100
0 of 90 found in topn= 100
0 of 6 found in topn= 100
0 of 42 found in topn= 100
0 of 60 found in topn= 100
0 of 17 found in topn= 100

In [131]:
user_profile = ratings_train.loc[ratings_train['userId'] == uid]
#user_profile = ratings_train[ratings_train['userId'] == uid]
user_profile

,userId,movieId,rating,timestamp
99893,671,260,5.0,1064891246
99973,671,4033,4.0,1065111954
99992,671,5816,4.0,1065111963
99930,671,1673,3.5,1063500961
99954,671,2997,4.5,1063503848
...,...,...,...,...
99908,671,1035,5.0,1065149492
99901,671,551,5.0,1063503908
99997,671,5995,4.0,1066793014
99919,671,1223,3.0,1064891236


In [23]:
u_p = ratings_train.loc[ratings_train['userId'] == 1]
print(len(u_p))
u_p

17


,userId,movieId,rating,timestamp
3,1,1129,2.0,1260759185
16,1,2294,2.0,1260759108
2,1,1061,3.0,1260759182
17,1,2455,2.5,1260759113
12,1,1953,4.0,1260759191
11,1,1405,1.0,1260759203
0,1,31,2.5,1260759144
6,1,1287,2.0,1260759187
14,1,2150,3.0,1260759194
13,1,2105,4.0,1260759139


In [25]:
user_profile_movie_list = u_p['movieId'].to_list()
user_m_index_list = []
tmdbId_list = []
for m in user_profile_movie_list:
    tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
    try:
        index = movieid_list.index(int(tmdbId))
        user_m_index_list.append(index)
        tmdbId_list.append(int(tmdbId))
    except ValueError:
        pass

In [26]:
user_m_index_list

[1094,
 2181,
 1031,
 2342,
 1844,
 1354,
 30,
 1242,
 2041,
 1996,
 1291,
 1218,
 1132,
 2084,
 1000,
 3549,
 1322]

In [152]:
#inferred_vector = model.infer_vector(tagged_text[1585].words)

#inferred_vector = model.infer_vector(text)
print(inferred_vector)
print(text.split())

[-0.03232411  0.06542593 -0.11860172 -0.02738422  0.08786373 -0.22847934
  0.04131952  0.31932786  0.1022366   0.07084516 -0.36353967 -0.15571275
  0.02770765  0.00269941 -0.27729848 -0.03791951  0.0838308  -0.12735762
 -0.06881724 -0.17752264  0.12573136 -0.15369149  0.32569042  0.26659033
 -0.05534959  0.01232429  0.15964577  0.03472032 -0.4914102   0.0380314
  0.04588491  0.07267144 -0.2519488  -0.05532525 -0.28103018 -0.00348903
  0.02023853  0.14145832  0.05940605 -0.01254564 -0.00198155 -0.01974271
  0.22153623  0.10047508  0.4180624   0.17659213  0.11772286 -0.24227077
  0.13943994  0.02141049]
['Toy', 'Story', '2', 'Andy', 'heads', 'off', 'to', 'Cowboy', 'Camp,', 'leaving', 'his', 'toys', 'to', 'their', 'own', 'devices.', 'Things', 'shift', 'into', 'high', 'gear', 'when', 'an', 'obsessive', 'toy', 'collector', 'named', 'Al', 'McWhiggen,', 'owner', 'of', "Al's", 'Toy', 'Barn', 'kidnaps', 'Woody.', "Andy's", 'toys', 'mount', 'a', 'daring', 'rescue', 'mission,', 'Buzz', 'Lightyear

In [171]:
test = df[df['title'] == 'Mission: Impossible']
test

,id,title,overview
540,954,Mission: Impossible,"When Ethan Hunt, the leader of a crack espiona..."


In [174]:
vec_u = model.dv[208]
#print(vec)
sim = model.dv.most_similar([vec_u],topn=10)

inferred_vector = model.infer_vector(tagged_text[208].words, steps=30, alpha=0.025)
sim3 = model.dv.most_similar([inferred_vector], topn=20)

print(sim)
indexes =[i[0] for i in sim]
for i in indexes:
    print(df['title'][i])

print("")

print(sim3)
indexes =[i[0] for i in sim3]
for i in indexes:
    print(df['title'][i])


[(208, 1.0000001192092896), (988, 0.9461885094642639), (4939, 0.9310348033905029), (977, 0.9279372096061707), (37292, 0.9215072393417358), (33017, 0.9209285974502563), (15498, 0.9153652787208557), (34522, 0.9128983020782471), (22571, 0.9114615321159363), (9695, 0.9062817096710205)]
Star Wars
Return of the Jedi
The Sword and the Sorcerer
The Empire Strikes Back
Gor
Kingsglaive: Final Fantasy XV
Asterix and the Vikings
Rogue One: A Star Wars Story
Star Wars: The Force Awakens
The Conqueror

[(208, 0.9564753174781799), (977, 0.8945705890655518), (988, 0.8773837089538574), (33017, 0.8724589347839355), (22571, 0.8557488918304443), (4939, 0.8529732823371887), (34522, 0.840722382068634), (37292, 0.8355525135993958), (15498, 0.8342955112457275), (25769, 0.8294712901115417), (9695, 0.8277966380119324), (7752, 0.8274527192115784), (2161, 0.8174904584884644), (26458, 0.8165868520736694), (15188, 0.8074105978012085), (17113, 0.8060462474822998), (5792, 0.8058078289031982), (11597, 0.80567955970764

In [169]:
cosine_similarity(inferred_vector.reshape(1, -1),vec_u.reshape(1, -1))

array([[0.942365]], dtype=float32)

In [163]:
# Toy Story, Spider-Man, The Avengers, Batman
mids = [0,4460,15159,497]
inferred_vector1 = model.infer_vector(tagged_text[0].words, steps=30, alpha=0.025)
inferred_vector2 = model.infer_vector(tagged_text[4460].words, steps=30, alpha=0.025)
inferred_vector3 = model.infer_vector(tagged_text[15159].words, steps=30, alpha=0.025)
inferred_vector4 = model.infer_vector(tagged_text[497].words, steps=30, alpha=0.025)


test_user_array = np.zeros((4,len(inferred_vector1)))
test_user_array[0] = inferred_vector1
test_user_array[1] = inferred_vector2
test_user_array[2] = inferred_vector3
test_user_array[3] = inferred_vector4

cos_sim = cosine_similarity(test_user_array, test_user_array)
print("Cos Sim: ")
print(cos_sim)

test_user = np.mean(test_user_array, axis=0)

sim = model.dv.most_similar([test_user], topn=len(model.dv))

#print(sim)
indexes =[i[0] for i in sim[:10]]
for i in indexes:
    print(df['title'][i])

#rank
rank_list = []
for mid in mids:
    rank = 0
    print("Mid: " + str(mid))
    for m_i, cos in sim:
        if m_i == mid:
            print("Rank: " + str(rank))
            break
        rank = rank + 1
        
    rank_list.append(rank)   

print("Max Rank = " + str(len(df)))
average_rank = sum(rank_list) / len(rank_list)
print("Average Rank of already seen movies: " + str(average_rank))

Cos Sim: 
[[1.         0.42783525 0.36287829 0.30135735]
 [0.42783525 1.         0.42588261 0.54596214]
 [0.36287829 0.42588261 1.         0.55471191]
 [0.30135735 0.54596214 0.55471191 1.        ]]
Batman
Wonder Woman
Detective Byomkesh Bakshy!
Lawnmower Man 2: Beyond Cyberspace
Freaky Deaky
Link
Captain Video, Master of the Stratosphere
Small Soldiers
Get Smart, Again!
Superman vs. The Elite
Mid: 0
Rank: 5074
Mid: 4460
Rank: 55
Mid: 15159
Rank: 16
Mid: 497
Rank: 196
Max Rank = 37713
Average Rank of already seen movies: 1335.25


In [153]:
rank_list

[20, 14, 20, 20]

In [165]:
# The Godfather, The Bourne Identity, The Pianist, Saving Pravite Rayn
inferred_vector1 = model.infer_vector(tagged_text[694].words, steps=30, alpha=0.025)
inferred_vector2 = model.infer_vector(tagged_text[4523].words, steps=30, alpha=0.025)
inferred_vector3 = model.infer_vector(tagged_text[5018].words, steps=30, alpha=0.025)
inferred_vector4 = model.infer_vector(tagged_text[1637].words, steps=30, alpha=0.025)


test_user_array = np.zeros((4,len(inferred_vector1)))
test_user_array[0] = inferred_vector1
test_user_array[1] = inferred_vector2
test_user_array[2] = inferred_vector3
test_user_array[3] = inferred_vector4

cos_sim = cosine_similarity(test_user_array, test_user_array)
print("Cos Sim: ")
print(cos_sim)

test_user = np.mean(test_user_array, axis=0)

sim = model.dv.most_similar([test_user], topn=len(model.dv))

print(sim[:10])
indexes =[i[0] for i in sim[:10]]
for i in indexes:
    print(df['title'][i])

#rank
rank_list = []
for mid in mids:
    rank = 0
    print("Mid: " + str(mid))
    for m_i, cos in sim:
        if m_i == mid:
            print("Rank: " + str(rank))
            break
        rank = rank + 1
        
    rank_list.append(rank)   

print("Max Rank = " + str(len(df)))
average_rank = sum(rank_list) / len(rank_list)
print("Average Rank of already seen movies: " + str(average_rank))

Cos Sim: 
[[1.         0.29128387 0.47421021 0.28500423]
 [0.29128387 1.         0.39641955 0.44367252]
 [0.47421021 0.39641955 1.         0.46195455]
 [0.28500423 0.44367252 0.46195455 1.        ]]
[(16829, 0.8986981511116028), (14813, 0.8905784487724304), (33350, 0.8882643580436707), (19358, 0.8848171830177307), (8604, 0.88371741771698), (6612, 0.8831021189689636), (9444, 0.8820332884788513), (3819, 0.8818320035934448), (20618, 0.877967119216919), (23359, 0.8763169050216675)]
Fate of a Man
Cornered
Chosen
Hornet's Nest
Act of Violence
Three Came Home
Trial on the Road
Casualties of War
The Look of Silence
The Diary of Anne Frank
Mid: 0
Rank: 36009
Mid: 4460
Rank: 27256
Mid: 15159
Rank: 14797
Mid: 497
Rank: 8926
Max Rank = 37713
Average Rank of already seen movies: 21747.0


In [54]:
ranks = []
for t in range(len(tagged_text)):
    inferred_vector = model.infer_vector(tagged_text[t].words)
    sims = model.dv.most_similar([inferred_vector], topn=100)
    try:
        rank = [docid for docid, sim in sims].index(t)
        ranks.append(rank)
    except ValueError:
        ranks.append(-1)

import collections

counter = collections.Counter(ranks)
print(counter)
print(counter[0]/len(tagged_text))

Counter({0: 40457, 1: 934, -1: 732, 2: 404, 3: 244, 4: 180, 5: 142, 6: 94, 7: 71, 8: 66, 9: 57, 13: 56, 10: 50, 11: 49, 16: 49, 12: 44, 14: 42, 17: 36, 19: 32, 20: 29, 15: 27, 21: 26, 18: 25, 26: 23, 24: 22, 25: 22, 23: 22, 31: 18, 22: 17, 38: 17, 28: 17, 27: 16, 30: 16, 29: 15, 39: 14, 42: 14, 59: 13, 33: 13, 54: 13, 35: 13, 36: 12, 37: 12, 34: 12, 48: 11, 51: 10, 55: 10, 32: 9, 49: 9, 44: 9, 84: 9, 63: 9, 43: 9, 56: 8, 40: 8, 41: 8, 57: 8, 81: 8, 85: 8, 50: 7, 69: 7, 46: 7, 58: 7, 91: 7, 47: 7, 72: 7, 53: 7, 92: 7, 66: 7, 73: 6, 78: 6, 64: 6, 52: 6, 45: 6, 83: 6, 60: 5, 77: 5, 96: 5, 70: 5, 71: 5, 82: 5, 95: 5, 65: 4, 67: 4, 98: 4, 80: 4, 68: 4, 76: 4, 61: 4, 79: 4, 97: 4, 87: 3, 93: 3, 90: 3, 74: 2, 89: 2, 88: 2, 99: 2, 75: 2, 86: 2, 62: 2, 94: 1})
0.9091256376261208


In [173]:
s = df[df['title'] == 'Star Wars']
s

,id,title,overview
208,11,Star Wars,Princess Leia is captured and held hostage by ...


In [21]:
# get only cos, sorted by index for model input
rec = recommendation_users
for k,l in rec.items():
    l.sort(key=lambda x:x[0])
    rec[k] = l

cos_inputs = {}
for uid in userid_list:
    u_reco = rec[uid]
    u_reco_list = list(map(list, zip(*u_reco)))
    u_reco_cos_array = np.array(u_reco_list[1])
    cos_inputs[uid] = u_reco_cos_array

In [23]:
len(cos_inputs[1])

44501

In [25]:
cos_inputs[1][:10]

array([0.32758981, 0.42638624, 0.49069268, 0.64956218, 0.2503736 ,
       0.69869506, 0.49557298, 0.47686332, 0.31756526, 0.59661818])

In [26]:
with open('doc2vec_vecsize_50_cos_inputs.pickle', 'wb') as f:
    pickle.dump(all_user_bitarrays,f, protocol=pickle.HIGHEST_PROTOCOL)